In [ ]:
import sys
import cv2
import numpy as np
import pytesseract
from math import ceil
import os 

print(sys.path)

#CV2 version:  4.8.0
#Numpy version:  1.25.2
#Tesseract version:  0.3.10

print("CV2 version: ", cv2.getVersionString())
print("Numpy version: ", np.__version__)
print("Tesseract version: ", pytesseract.__version__)

In [ ]:
# Choose image

output_folder = "./outputStages/Pipeline_0/"

#input_image = "./data/computer_generated_images/Sudoku_puzzle_hard_for_brute_force.jpg"  # https://commons.wikimedia.org/wiki/File:Sudoku_puzzle_hard_for_brute_force.jpg
#output_basename =  "Sudoku_puzzle_hard_for_brute_force"

input_image = "./data/computer_generated_images/Sudoku_Puzzle_by_L2G-20050714_standardized_layout.svg.png"  # https://upload.wikimedia.org/wikipedia/commons/e/e0/Sudoku_Puzzle_by_L2G-20050714_standardized_layout.svg
output_basename =  "Sudoku_Puzzle_by_L2G-20050714_standardized_layout"

In [ ]:
def printoutStage(stage_no, stage_img):
    global output_folder, output_basename
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print("Made designated output folder")
    cv2.imwrite(output_folder + output_basename + "-stage-" + str(stage_no) +".png", stage_img)

In [ ]:
# Load image as gray scale 
image = cv2.imread(input_image)
img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# ----------------
printoutStage(1, img_gray)

In [ ]:
# Gaussian Blur
image = img_gray
img_blur = cv2.GaussianBlur(img_gray,(5,5),0)

# ----------------
printoutStage(2, img_blur)

In [ ]:
# Adaptive Threshold

image = img_blur 

def apply_adaptive_threshold(image, block_size=11, constant_value=2):
    v1 = cv2.ADAPTIVE_THRESH_MEAN_C
    v2 = cv2.ADAPTIVE_THRESH_GAUSSIAN_C
    # Apply adaptive thresholding
    thresholded_image = cv2.adaptiveThreshold(
        image, 255, v2,
        cv2.THRESH_BINARY, block_size, constant_value
    )
    return thresholded_image

# Specify block size and constant value for adaptive thresholding
block_size = 11
constant_value = 2

# Apply adaptive thresholding to the image
img_thrsh = apply_adaptive_threshold(image, block_size, constant_value)

# ----------------
printoutStage(3, img_thrsh)

In [ ]:
# We start by finding contours in the thresholded image:

image = img_thrsh

contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#print(len(contours), contours[0])
#print(hierarchy)

img_contours = image.copy()
img_contours_color = cv2.cvtColor(img_contours, cv2.COLOR_GRAY2RGB)
cv2.drawContours(image=img_contours_color, contours=contours, 
  contourIdx = -1, color = (0,255,0), thickness=3, lineType=cv2.LINE_AA)

# ----------------
printoutStage(4, img_contours_color)

In [ ]:
# create the 4-corner approximations for all rectangle contours
image = img_thrsh

approxes = []
for idx, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    peri = cv2.arcLength(contour,True)
    approx = cv2.approxPolyDP(contour,0.02*peri,True)
    if len(approx)==4: 
        approxes.append(approx)
print("Number of contour approxes: ( should be at least 82 )", len(approxes))

# plot them for me
img_contours = image.copy()
img_contours_color = cv2.cvtColor(img_contours, cv2.COLOR_GRAY2RGB)
cv2.drawContours(image=img_contours_color, contours=approxes, 
    contourIdx = -1 , color = (0,255,0), thickness=3, lineType=cv2.LINE_AA)

# ----------------
printoutStage(5, img_contours_color)

In [ ]:
# find the contour approx area which has exactly 81 repetitions 
approx_areas = np.array([])
approx_areas_idxs = []
for idx, approx in enumerate(approxes):
    area = cv2.contourArea(approx)
    x = np.where( (approx_areas >= 0.85*area) & (approx_areas <= 1.15*area) )
    # print(idx, area, approx_areas, x, len(x))
    # x contains at least idx
    if len(x[0]) == 0:
        print("First time we met this area size")
        approx_areas = np.append(approx_areas,area)
        approx_areas_idxs.append([idx])    
    elif len(x[0]) == 1:
        #print("Not the first time we met this area size, add index to area size index list ")
        approx_areas_idxs[x[0][0]].append(idx)
    elif len(x[0]) > 1:
        print("This is a problem!")    
    
# one length should be 81
approx_areas_idxs_lens = np.array([])
for idxs in approx_areas_idxs:
    approx_areas_idxs_lens = np.append(approx_areas_idxs_lens, len(idxs))

print("approx areas:", approx_areas)
print("approx areas idxs:", approx_areas_idxs)
print("approx areas idxs lens:", approx_areas_idxs_lens)

x = np.where(approx_areas_idxs_lens == 81)
if len(x[0])==0 or len(x[0])>1:
    print("\n\nERROR did not find the correct small squares")
else:
    sq_idxs = sorted(approx_areas_idxs[x[0][0]])
    print("\n\nALL OK ! idx of small squares in \"approxes\" is:", sq_idxs)
    # work only with the little squares blob approximations
    approxes_sq = np.array(approxes)[sq_idxs].tolist()

In [ ]:
# Utility function, needed later
# cluster a set of values into clusters of size k
# by sorting the data set and then grouping into the cluster size 
def my_cluster(x, k=2):
   # y = sorted(x)
   idx = sorted(range(len(x)), key=lambda j: x[j])
   y = [x[j] for j in idx]

   # sanity check todo
   clusters = []
   means = []
   idx_clustered = []
   for i in range(ceil(len(y)/k)):
      clusters.append(y[i*k:(i+1)*k])
      idx_clustered.append(idx[i*k:(i+1)*k])
      means.append(np.mean(clusters[-1]))
   return clusters, means, idx_clustered 

# unit test of function 
print(my_cluster([10,10.5,8.1,7.9,6.3,5.9,14.2,14.3,2,1]))
print(my_cluster([10,10.5,8.1,7.9,6.3,5.9,14.2,14.3,2,1],3))

In [ ]:
# Now we need to order the approxes by coordinates, to get the numbers in right order !
boxes = []
boxes_ulc = []  # upper left corner only
for idx, approx in enumerate(approxes_sq):
    approx = np.array(approx)
    boxes.append([ my_cluster(approx[:,0,0])[1] , my_cluster(approx[:,0,1])[1] ])   # [ [xmin, xmax] , [ymin,y_max] ] 
    boxes_ulc.append([boxes[-1][0][0],boxes[-1][1][0]])  # [ [xmin, ymin ] ] 
print(len(boxes), boxes, "\n\n", len(boxes_ulc), boxes_ulc, "\n")

In [ ]:
# We need to sort the boxes
# We need to cluster y in 9 rows
row_clusters, row_y_coords, row_idxs = my_cluster(np.array(boxes_ulc)[:,1].tolist(),9)

# Safety checks
for i, c in enumerate(row_clusters):
    if len(c) != 9:
       print("ERROR not 9") 
if len(row_y_coords) != 9:
    print("ERROR not 9")

print("row_clusters:", row_clusters)
print("row_y_coords:", row_y_coords)
print("row_idxs:", row_idxs, "\n")

row_idxs_x_sorted = []
# I need to sort the x also now, so inside every idx cluster, I need to resort 
for i, idx_cluster in enumerate(row_idxs):
    # e.g. idx_cluster = [63, 64, 65, 66, 67, 68, 69, 70, 71]
    
    # build the x coordinate list corresponding to this cluster 
    x_coords = []
    for j in idx_cluster:
        x_coords.append(np.array(boxes_ulc)[j,0])
    print("x_coords:", x_coords)        
    
    # I need x_coords indices sorted
    idx_x_coords_sorted = sorted(range(len(x_coords)), key = lambda j: x_coords[j])
    x_coords_sorted = [x_coords[j] for j in idx_x_coords_sorted]

    # print for visual inspection
    print("x_coords_sorted:", x_coords_sorted)
    print("New sorting of index cluster:",np.array(idx_cluster)[idx_x_coords_sorted].tolist())
    
    row_idxs_x_sorted.append(np.array(idx_cluster)[idx_x_coords_sorted].tolist())
    print(row_idxs_x_sorted)

In [ ]:
print(row_idxs_x_sorted) # sorted version

for i in range(len(approxes_sq)):
    approxes_sq[i] = np.array(approxes_sq[i])

# Single test
# img_contours_color4 = cv2.cvtColor(img_contours_2, cv2.COLOR_GRAY2RGB)
# cv2.drawContours(image=img_contours_color4, contours = approxes_sq, 
#     contourIdx = row_idxs_x_sorted[2][7] , color = (0,255,0), thickness=3, lineType=cv2.LINE_AA)
# cv2.destroyAllWindows()
# cv2.imshow('image6', img_contours_color4)
# cv2.waitKey(1)

# full visual inspection from stiching back
PSIZE = -1
img_all = np.array([])
for idxs in row_idxs_x_sorted:
    img_row = np.array([])
    for idx in idxs: 
        y_min = int(boxes[idx][1][0])
        y_max = int(boxes[idx][1][1])
        x_min = int(boxes[idx][0][0])
        x_max = int(boxes[idx][0][1])
        img_new = img_gray[y_min:y_max, x_min:x_max]
        if (PSIZE == -1):
            PSIZE = abs(max(x_max-x_min, y_max-y_min)*2)
        v0 = int(PSIZE - img_new.shape[0])
        v1 = int(PSIZE - img_new.shape[1])
        img_new_padded = np.pad(img_new, ((0, v0), (0, v1)), 'constant', constant_values=100)
        print("Shapes:\t", img_new.shape, img_new_padded.shape, img_row.shape)
        if len(img_row):
            img_row = np.concatenate( (img_row, img_new_padded), axis = 1 ) # stack horizontally
        else:
            img_row = img_new_padded
    if len(img_all):
        img_all = np.concatenate((img_all, img_row), axis=0) # stack vertically
    else:
        img_all = img_row


# ----------------
stage = 6
img_stage = img_all
cv2.imwrite(output_folder + output_basename + "-stage-" + str(stage) +".png", img_stage)

In [ ]:
# Get each little square box and OCR the digit;
# But also plot the visual inspection segmentation
# print(boxes)

PSIZE = -1
img_all = np.array([])
img_row = np.array([])
ctr = 0

SUDOKU_MATRIX = np.zeros([9,9])
ii = 0
for idxs in row_idxs_x_sorted:
    jj = 0
    for idx in idxs: 
        y_min = int(boxes[idx][1][0])
        y_max = int(boxes[idx][1][1])
        x_min = int(boxes[idx][0][0])
        x_max = int(boxes[idx][0][1])
        
        #ROI = img_gray[y_min:y_max, x_min:x_max]
        ROI = img_gray[y_min:y_max, x_min:x_max]
        print("Shape:\t", ROI.shape)
        ocr_result = ""

        # code for segmentation stiching 
        # ------------------------------------------------
        if (PSIZE == -1):
            PSIZE = abs(max(x_max-x_min, y_max-y_min)*2)
        v0 = int(PSIZE - ROI.shape[0])
        v1 = int(PSIZE - ROI.shape[1])
        ROI_padded = np.pad(ROI, ((0, v0), (0, v1)), 'constant', constant_values=100)
        if len(img_row):
            img_row = np.concatenate( (img_row, ROI_padded), axis = 1 ) # stack horizontally
        else:
            img_row = ROI_padded
        ctr = ctr + 1
        if ctr == 9:
          ctr = 0
          if len(img_all):
            img_all = np.concatenate((img_all, img_row), axis=0) # stack vertically
          else:
            img_all = img_row
          img_row = np.array([]) 
        # ------------------------------------------------

        # Threshold analysis, because of tesseract identifying  characters where there are none
        # Calculate the percentage of non-white pixels
        #non_white_pixels = cv2.countNonZero(ROI)
        non_white_pixels = ( ROI.flatten() < 155 ).sum()
        total_pixels = ROI.shape[0] * ROI.shape[1]
        non_white_ratio = non_white_pixels / total_pixels
        print("Non White Ratio Result: ", idx, non_white_ratio)

        # Compare the non-white ratio with the threshold
        if non_white_ratio >= 0.025:
            _, thresholded = cv2.threshold(ROI, 150, 255, cv2.THRESH_BINARY)

            # Specify your whitelist of characters. We want only digits:
            ocr_result = pytesseract.image_to_string(thresholded, config = r'--psm 10 -c tessedit_char_whitelist=123456789')
            ocr_result = ocr_result.strip()
           
            print("OCR Result: ", idx, ocr_result)
            SUDOKU_MATRIX[ii,jj] =  int(ocr_result)
        jj = jj + 1
    
    ii = ii + 1    

#------------------
printoutStage(10,img_all)

####################
# # Page segmentation modes:
#  0    Orientation and script detection (OSD) only.
#  1    Automatic page segmentation with OSD.
#  2    Automatic page segmentation, but no OSD, or OCR.
#  3    Fully automatic page segmentation, but no OSD. (Default)
#  4    Assume a single column of text of variable sizes.
#  5    Assume a single uniform block of vertically aligned text.
#  6    Assume a single uniform block of text.
#  7    Treat the image as a single text line.
#  8    Treat the image as a single word.
#  9    Treat the image as a single word in a circle.
# 10    Treat the image as a single character.
# 11    Sparse text. Find as much text as possible in no particular order.
# 12    Sparse text with OSD.
# 13    Raw line. Treat the image as a single text line,
#                        bypassing hacks that are Tesseract-specific.      

In [ ]:
print("SUDOKU MATRIX:\n", SUDOKU_MATRIX)